# Capítulo 4: Primitivas SIMD e Execução Branchless

## Introdução

**SIMD** (Single Instruction, Multiple Data) permite processar múltiplos valores com uma única instrução de CPU. O DuckDB aproveita extensivamente SIMD para acelerar operações vetorizadas.

### Objetivos deste capítulo:
1. Entender o que é SIMD e como funciona
2. Aprender sobre execução branchless (sem saltos condicionais)
3. Comparar performance com e sem SIMD
4. Implementar operações SIMD-friendly em Python/NumPy

In [ ]:
!pip install duckdb pandas numpy matplotlib seaborn -q

In [ ]:
import duckdb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
from typing import List

sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

## 4.1 O que é SIMD?

SIMD permite que uma única instrução opere em múltiplos dados simultaneamente.

### Processamento Escalar (tradicional):
```
Instrução 1: a[0] + b[0] = c[0]
Instrução 2: a[1] + b[1] = c[1]
Instrução 3: a[2] + b[2] = c[2]
Instrução 4: a[3] + b[3] = c[3]
```

### Processamento SIMD (vetorial):
```
Instrução 1: [a[0], a[1], a[2], a[3]] + [b[0], b[1], b[2], b[3]] = [c[0], c[1], c[2], c[3]]
```

**Uma instrução, 4 operações!**

### Extensões SIMD em CPUs modernas:
- **SSE/SSE2**: 128 bits (4 floats ou 2 doubles)
- **AVX/AVX2**: 256 bits (8 floats ou 4 doubles)
- **AVX-512**: 512 bits (16 floats ou 8 doubles)

In [ ]:
# Verificar suporte SIMD do NumPy
print("Configuração NumPy:")
print(np.show_config())

In [ ]:
# Demonstração: Operação escalar vs vetorial

def scalar_add(a: np.ndarray, b: np.ndarray) -> np.ndarray:
    """Soma elemento por elemento (escalar) - LENTO"""
    result = np.zeros_like(a)
    for i in range(len(a)):
        result[i] = a[i] + b[i]
    return result

def vectorized_add(a: np.ndarray, b: np.ndarray) -> np.ndarray:
    """Soma vetorizada (SIMD-friendly) - RÁPIDO"""
    return a + b  # NumPy usa SIMD automaticamente!

# Benchmark
size = 1_000_000
a = np.random.randn(size).astype(np.float64)
b = np.random.randn(size).astype(np.float64)

# Escalar
start = time.perf_counter()
result_scalar = scalar_add(a, b)
scalar_time = time.perf_counter() - start

# Vetorizado
start = time.perf_counter()
result_vec = vectorized_add(a, b)
vec_time = time.perf_counter() - start

print(f"Soma de {size:,} elementos:")
print(f"  Escalar:    {scalar_time*1000:.2f} ms")
print(f"  Vetorizado: {vec_time*1000:.4f} ms")
print(f"  Speedup:    {scalar_time/vec_time:.0f}x")

## 4.2 Branch Prediction e o Problema dos Saltos

CPUs modernas usam **pipeline** para executar instruções em paralelo. Quando há um `if`, a CPU precisa "adivinhar" qual caminho será tomado.

```
Pipeline da CPU:
┌─────┬─────┬─────┬─────┬─────┐
│Fetch│Decode│Execute│Memory│Write│
└─────┴─────┴─────┴─────┴─────┘
   ↓     ↓      ↓       ↓      ↓
  I1    I2     I3      I4     I5   (5 instruções simultâneas)
```

### Se a previsão falha:
```
if (condition) → CPU adivinha TRUE
   ... executa código do TRUE ...
   OOPS! Era FALSE!
   → FLUSH do pipeline (descarta trabalho)
   → ~15-20 ciclos perdidos!
```

**Dados aleatórios = 50% de erro = DESASTRE!**

In [ ]:
# Demonstração: Impacto de branch misprediction

def with_branches(data: np.ndarray, threshold: int) -> np.ndarray:
    """Implementação com branches (if/else)"""
    result = np.zeros_like(data)
    for i in range(len(data)):
        if data[i] > threshold:  # BRANCH!
            result[i] = 1
        else:
            result[i] = 0
    return result

def branchless(data: np.ndarray, threshold: int) -> np.ndarray:
    """Implementação branchless (sem if/else)"""
    return (data > threshold).astype(np.int64)  # Comparação vetorizada!

# Teste com dados ORDENADOS (branch prediction funciona bem)
sorted_data = np.sort(np.random.randint(0, 100, 100_000))

# Teste com dados ALEATÓRIOS (branch prediction falha muito)
random_data = np.random.randint(0, 100, 100_000)

threshold = 50

print("Comparação: Branch vs Branchless\n")

# Dados ordenados
start = time.perf_counter()
_ = with_branches(sorted_data, threshold)
sorted_branch_time = time.perf_counter() - start

start = time.perf_counter()
_ = branchless(sorted_data, threshold)
sorted_branchless_time = time.perf_counter() - start

print(f"Dados ORDENADOS (previsão fácil):")
print(f"  Com branch:    {sorted_branch_time*1000:.2f} ms")
print(f"  Branchless:    {sorted_branchless_time*1000:.4f} ms")

# Dados aleatórios
start = time.perf_counter()
_ = with_branches(random_data, threshold)
random_branch_time = time.perf_counter() - start

start = time.perf_counter()
_ = branchless(random_data, threshold)
random_branchless_time = time.perf_counter() - start

print(f"\nDados ALEATÓRIOS (previsão difícil):")
print(f"  Com branch:    {random_branch_time*1000:.2f} ms")
print(f"  Branchless:    {random_branchless_time*1000:.4f} ms")
print(f"\n→ Branchless é consistentemente rápido independente dos dados!")

## 4.3 Operações Branchless com Máscaras

A técnica branchless usa **máscaras de bits** para selecionar resultados sem saltos.

```cpp
// COM branch (lento com dados aleatórios)
if (val > 50) result = 1; else result = 0;

// SEM branch (SIMD-friendly)
mask = (val > 50);  // Gera 0 ou 1 (ou 0xFFFF...)
result = mask & 1;  // Aplica máscara
```

In [ ]:
# Demonstração visual de operações com máscaras

data = np.array([10, 60, 30, 80, 20, 90, 40, 70], dtype=np.int64)
threshold = 50

# Criar máscara de comparação
mask = data > threshold

print("Dados:         ", data)
print("Threshold:     ", threshold)
print("Máscara (bool):", mask.astype(int))
print("")

# Usar máscara para diferentes operações
# Operação 1: Selecionar valor ou zero
result_or_zero = data * mask
print("data * mask (valor ou 0):  ", result_or_zero)

# Operação 2: Selecionar entre dois valores
value_true = 100
value_false = -1
result_select = np.where(mask, value_true, value_false)
print(f"where(mask, {value_true}, {value_false}):   ", result_select)

# Operação 3: Contar elementos que passam
count = mask.sum()
print(f"Contagem (> {threshold}):       ", count)

## 4.4 SIMD em Comparações e Filtros

In [ ]:
# Benchmark: Diferentes operações de comparação

size = 10_000_000
data = np.random.randint(0, 100, size, dtype=np.int64)
threshold = 50

operations = {
    'Maior que (>)': lambda: data > threshold,
    'Igual (==)': lambda: data == threshold,
    'Entre (20 < x < 80)': lambda: (data > 20) & (data < 80),
    'Divisível por 3': lambda: data % 3 == 0,
    'Máximo': lambda: np.maximum(data, threshold),
    'Mínimo': lambda: np.minimum(data, threshold),
    'Clip (20-80)': lambda: np.clip(data, 20, 80),
}

print(f"Benchmark de operações SIMD em {size:,} elementos:\n")

results = {}
for name, op in operations.items():
    times = []
    for _ in range(10):
        start = time.perf_counter()
        _ = op()
        times.append(time.perf_counter() - start)
    mean_time = np.mean(times) * 1000
    results[name] = mean_time
    throughput = size / (np.mean(times) * 1e9)  # Bilhões de ops/seg
    print(f"{name:25} {mean_time:6.2f} ms ({throughput:.1f} GOP/s)")

In [ ]:
# Visualização
fig, ax = plt.subplots(figsize=(12, 5))

names = list(results.keys())
times = list(results.values())

colors = plt.cm.viridis(np.linspace(0.2, 0.8, len(names)))
bars = ax.barh(names, times, color=colors)

ax.set_xlabel('Tempo (ms)')
ax.set_title(f'Performance de Operações SIMD ({size:,} elementos)')

# Adicionar valores nas barras
for bar, t in zip(bars, times):
    ax.text(bar.get_width() + 0.1, bar.get_y() + bar.get_height()/2, 
            f'{t:.2f}ms', va='center', fontsize=9)

plt.tight_layout()
plt.show()

## 4.5 Agregações SIMD

In [ ]:
# Comparação: Agregação escalar vs SIMD

def scalar_sum(data: np.ndarray) -> float:
    """Soma escalar - LENTO"""
    total = 0
    for x in data:
        total += x
    return total

def simd_sum(data: np.ndarray) -> float:
    """Soma SIMD via NumPy"""
    return np.sum(data)

# Benchmark
size = 10_000_000
data = np.random.randn(size)

# Escalar
start = time.perf_counter()
result_scalar = scalar_sum(data)
scalar_time = time.perf_counter() - start

# SIMD
start = time.perf_counter()
result_simd = simd_sum(data)
simd_time = time.perf_counter() - start

print(f"Soma de {size:,} elementos:")
print(f"  Escalar: {scalar_time*1000:.2f} ms")
print(f"  SIMD:    {simd_time*1000:.4f} ms")
print(f"  Speedup: {scalar_time/simd_time:.0f}x")
print(f"\nResultado escalar: {result_scalar:.6f}")
print(f"Resultado SIMD:    {result_simd:.6f}")

## 4.6 Padrões Comuns do DuckDB

O DuckDB implementa várias operações usando padrões SIMD-friendly.

In [ ]:
# Padrão 1: Filtro com agregação (comum em queries)
# SELECT SUM(valor) FROM tabela WHERE condicao

def filter_then_sum_scalar(data: np.ndarray, threshold: int) -> float:
    """Filtra e soma - versão escalar"""
    total = 0
    for x in data:
        if x > threshold:
            total += x
    return total

def filter_then_sum_simd(data: np.ndarray, threshold: int) -> float:
    """Filtra e soma - versão SIMD"""
    mask = data > threshold
    return np.sum(data * mask)  # Branchless!

def filter_then_sum_simd_v2(data: np.ndarray, threshold: int) -> float:
    """Filtra e soma - versão SIMD alternativa"""
    return np.sum(data[data > threshold])  # Usa indexação booleana

# Benchmark
size = 1_000_000
data = np.random.randint(0, 100, size, dtype=np.int64)
threshold = 50

methods = {
    'Escalar (loop + if)': lambda: filter_then_sum_scalar(data, threshold),
    'SIMD (mask * sum)': lambda: filter_then_sum_simd(data, threshold),
    'SIMD (boolean index)': lambda: filter_then_sum_simd_v2(data, threshold),
}

print(f"SUM(x) WHERE x > {threshold} em {size:,} elementos:\n")

for name, method in methods.items():
    times = []
    for _ in range(10):
        start = time.perf_counter()
        result = method()
        times.append(time.perf_counter() - start)
    mean_time = np.mean(times) * 1000
    print(f"{name:25} {mean_time:8.3f} ms | Resultado: {result:,}")

In [ ]:
# Padrão 2: CASE WHEN (muito usado em SQL)
# SELECT CASE WHEN x > 50 THEN 'Alto' ELSE 'Baixo' END

def case_when_scalar(data: np.ndarray) -> np.ndarray:
    """CASE WHEN - versão escalar com branches"""
    result = np.empty(len(data), dtype=object)
    for i, x in enumerate(data):
        if x > 50:
            result[i] = 'Alto'
        else:
            result[i] = 'Baixo'
    return result

def case_when_simd(data: np.ndarray) -> np.ndarray:
    """CASE WHEN - versão SIMD branchless"""
    return np.where(data > 50, 'Alto', 'Baixo')

# Benchmark
size = 100_000
data = np.random.randint(0, 100, size, dtype=np.int64)

print(f"CASE WHEN em {size:,} elementos:\n")

start = time.perf_counter()
_ = case_when_scalar(data)
scalar_time = time.perf_counter() - start

start = time.perf_counter()
_ = case_when_simd(data)
simd_time = time.perf_counter() - start

print(f"Escalar: {scalar_time*1000:.2f} ms")
print(f"SIMD:    {simd_time*1000:.4f} ms")
print(f"Speedup: {scalar_time/simd_time:.0f}x")

## 4.7 DuckDB: Verificando Uso de SIMD

In [ ]:
# Conectar ao DuckDB
con = duckdb.connect(':memory:')

# Criar tabela grande
con.execute("""
    CREATE TABLE benchmark AS
    SELECT 
        i AS id,
        random() * 100 AS valor,
        (random() * 1000)::INTEGER AS quantidade
    FROM generate_series(1, 10000000) AS t(i)
""")

print("Tabela criada com 10 milhões de linhas")

In [ ]:
# Benchmark de operações típicas
queries = {
    'SUM simples': 'SELECT SUM(valor) FROM benchmark',
    'SUM com filtro': 'SELECT SUM(valor) FROM benchmark WHERE valor > 50',
    'COUNT com filtro': 'SELECT COUNT(*) FROM benchmark WHERE valor > 50 AND quantidade > 500',
    'CASE WHEN': 'SELECT SUM(CASE WHEN valor > 50 THEN quantidade ELSE 0 END) FROM benchmark',
    'Múltiplas agregações': 'SELECT SUM(valor), AVG(valor), MIN(valor), MAX(valor) FROM benchmark',
}

print("Performance DuckDB (SIMD otimizado):\n")

duck_results = {}
for name, query in queries.items():
    times = []
    for _ in range(5):
        start = time.perf_counter()
        con.execute(query).fetchall()
        times.append(time.perf_counter() - start)
    mean_time = np.mean(times) * 1000
    duck_results[name] = mean_time
    print(f"{name:25} {mean_time:8.2f} ms")

In [ ]:
# Ver plano de execução com profiling
print("\n=== EXPLAIN ANALYZE ===")
plan = con.execute("""
    EXPLAIN ANALYZE
    SELECT SUM(CASE WHEN valor > 50 THEN quantidade ELSE 0 END) 
    FROM benchmark
""").df()
print(plan.to_string())

## 4.8 Exercícios Práticos

### Exercício 1: Implementar operação branchless para múltiplos thresholds

In [ ]:
def categorize_branchless(data: np.ndarray) -> np.ndarray:
    """
    Categoriza dados em 4 faixas sem usar branches:
    - < 25: 0 (Baixo)
    - 25-50: 1 (Médio-Baixo)
    - 50-75: 2 (Médio-Alto)
    - >= 75: 3 (Alto)
    """
    # Usando comparações vetorizadas e soma de máscaras
    result = np.zeros_like(data, dtype=np.int32)
    result += (data >= 25).astype(np.int32)
    result += (data >= 50).astype(np.int32)
    result += (data >= 75).astype(np.int32)
    return result

# Teste
test_data = np.array([10, 30, 55, 80, 25, 50, 75, 100])
categories = categorize_branchless(test_data)

category_names = ['Baixo', 'Médio-Baixo', 'Médio-Alto', 'Alto']
print("Dados:      ", test_data)
print("Categorias: ", categories)
print("Nomes:      ", [category_names[c] for c in categories])

### Exercício 2: Comparar performance de diferentes abordagens

In [ ]:
def benchmark_approaches(size: int = 1_000_000):
    """Compara diferentes abordagens para a mesma operação"""
    data = np.random.randint(0, 100, size, dtype=np.int64)
    
    # Operação: contar elementos > 50 e somar seus valores
    
    # Abordagem 1: Loop Python
    def python_loop():
        count = 0
        total = 0
        for x in data:
            if x > 50:
                count += 1
                total += x
        return count, total
    
    # Abordagem 2: NumPy com indexação
    def numpy_indexing():
        filtered = data[data > 50]
        return len(filtered), np.sum(filtered)
    
    # Abordagem 3: NumPy branchless
    def numpy_branchless():
        mask = data > 50
        return np.sum(mask), np.sum(data * mask)
    
    approaches = {
        'Python loop': python_loop,
        'NumPy indexing': numpy_indexing,
        'NumPy branchless': numpy_branchless,
    }
    
    print(f"Benchmark: contar e somar elementos > 50 em {size:,} elementos\n")
    
    for name, func in approaches.items():
        # Verificar resultado
        count, total = func()
        
        # Medir tempo
        times = []
        n_iter = 1 if 'Python' in name else 10
        for _ in range(n_iter):
            start = time.perf_counter()
            func()
            times.append(time.perf_counter() - start)
        
        mean_time = np.mean(times) * 1000
        print(f"{name:20} {mean_time:10.3f} ms | count={count:,}, sum={total:,}")

benchmark_approaches()

## 4.9 Resumo do Capítulo

### SIMD e Branchless - Conceitos Chave

| Técnica | Benefício | Quando Usar |
|---------|-----------|-------------|
| **SIMD** | Processa N valores por instrução | Operações em arrays |
| **Branchless** | Evita pipeline stalls | Condições com dados aleatórios |
| **Máscaras** | Seleção sem branches | Filtros, CASE WHEN |
| **Agregações vetorizadas** | Reduz loop overhead | SUM, AVG, COUNT |

### Principais Takeaways:

1. **SIMD processa múltiplos dados** com uma única instrução
2. **Branch misprediction** pode custar 15-20 ciclos de CPU
3. **Operações branchless** usam máscaras em vez de if/else
4. **NumPy** usa SIMD automaticamente para operações vetorizadas
5. **DuckDB** implementa todas essas otimizações internamente

In [ ]:
con.close()